In [ ]:
pip install pycryptodome cryptography rsa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 29.5 MB/s eta 0:00:00


In [ ]:
import os
import json
import base64
import hashlib
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Util.Padding import pad, unpad
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes

class DiffieHellman:
    def __init__(self):
        self.parameters = dh.generate_parameters(generator=2, key_size=512)
        self.keys = {}

    def generate_private_key(self, user):
        private_key = self.parameters.generate_private_key()
        self.keys[user] = private_key
        return private_key

    def compute_shared_key(self, user1, user2):
        if user1 not in self.keys or user2 not in self.keys:
            raise ValueError("Both users must have private keys")
        shared_key = self.keys[user1].exchange(self.keys[user2].public_key())
        derived_key = HKDF(algorithm=hashes.SHA256(), length=16, salt=None, info=None).derive(shared_key)
        return derived_key

class KeyRevocation:
    def __init__(self):
        self.revoked_keys = set()

    def revoke_key(self, user):
        self.revoked_keys.add(user)
        return f"Key for {user} has been revoked."

    def is_revoked(self, user):
        return user in self.revoked_keys

db = {}  # Stores keys (for demonstration only, use secure storage in real systems)
dh = DiffieHellman()
revocation_manager = KeyRevocation()

def generate_symmetric_key():
    user = input("Enter username: ").strip()
    if revocation_manager.is_revoked(user):
        print("[-] This user's key has been revoked.")
        return
    key = os.urandom(32)
    db[f'{user}_symmetric_key'] = key
    print(f"[+] AES Symmetric Key Generated for {user}.")
    print(f"Key: {base64.b64encode(key).decode()}")

def encrypt_decrypt_aes():
    user = input("Enter username: ").strip()
    if revocation_manager.is_revoked(user):
        print("[-] This user's key has been revoked.")
        return
    key_key = f'{user}_symmetric_key'
    if key_key not in db:
        print("[-] No AES key found for this user. Generate a key first.")
        return

    key = db[key_key]
    cipher = AES.new(key, AES.MODE_CBC, iv := os.urandom(16))
    plaintext = input("Enter message to encrypt: ").encode()
    ciphertext = cipher.encrypt(pad(plaintext, AES.block_size))
    print("[+] Encrypted Message:", base64.b64encode(iv + ciphertext).decode())

    # Decryption
    cipher = AES.new(key, AES.MODE_CBC, iv)
    decrypted = unpad(cipher.decrypt(ciphertext), AES.block_size)
    print("[+] Decrypted Message:", decrypted.decode())

def generate_rsa_keys():
    user = input("Enter username: ").strip()
    if revocation_manager.is_revoked(user):
        print("[-] This user's key has been revoked.")
        return
    key = RSA.generate(2048)
    db[f'{user}_rsa_private'] = key.export_key()
    db[f'{user}_rsa_public'] = key.publickey().export_key()
    print(f"[+] RSA Key Pair Generated for {user}.")
    print("Public Key:")
    print(db[f'{user}_rsa_public'].decode())
    print("Private Key:")
    print(db[f'{user}_rsa_private'].decode())

def encrypt_decrypt_rsa():
    user = input("Enter username: ").strip()
    if revocation_manager.is_revoked(user):
        print("[-] This user's key has been revoked.")
        return
    public_key_key = f'{user}_rsa_public'
    private_key_key = f'{user}_rsa_private'
    if public_key_key not in db or private_key_key not in db:
        print("[-] No RSA keys found for this user. Generate keys first.")
        return

    message = input("Enter message to encrypt: ").encode()
    public_key = RSA.import_key(db[public_key_key])
    cipher_rsa = PKCS1_OAEP.new(public_key)
    encrypted = cipher_rsa.encrypt(message)
    print("[+] Encrypted Message:", base64.b64encode(encrypted).decode())

    # Decryption
    private_key = RSA.import_key(db[private_key_key])
    cipher_rsa = PKCS1_OAEP.new(private_key)
    decrypted = cipher_rsa.decrypt(encrypted)
    print("[+] Decrypted Message:", decrypted.decode())

def diffie_hellman_exchange():
    user1 = input("Enter first user: ")
    user2 = input("Enter second user: ")
    if revocation_manager.is_revoked(user1) or revocation_manager.is_revoked(user2):
        print("[-] One or both users have revoked keys.")
        return
    dh.generate_private_key(user1)
    dh.generate_private_key(user2)
    shared_key = dh.compute_shared_key(user1, user2)
    print("Shared Key (Diffie-Hellman):", base64.b64encode(shared_key).decode())

def revoke_key():
    user = input("Enter username: ").strip()
    print(revocation_manager.revoke_key(user))

def main():
    options = {
        '1': generate_symmetric_key,
        '2': encrypt_decrypt_aes,
        '3': generate_rsa_keys,
        '4': encrypt_decrypt_rsa,
        '5': diffie_hellman_exchange,
        '6': revoke_key,
        '7': exit
    }
    while True:
        print("\n1. Generate Symmetric Key")
        print("2. Encrypt & Decrypt Message (AES)")
        print("3. Generate RSA Keys")
        print("4. Encrypt & Decrypt Message (RSA)")
        print("5. Perform Diffie-Hellman Key Exchange")
        print("6. Revoke Key")
        print("7. Exit")
        choice = input("Select an option: ")
        if choice == '7':
            print("Exiting...")
            break
        options.get(choice, lambda: print("[-] Invalid choice."))()

if __name__ == "__main__":
    main()



1. Generate Symmetric Key
2. Encrypt & Decrypt Message (AES)
3. Generate RSA Keys
4. Encrypt & Decrypt Message (RSA)
5. Perform Diffie-Hellman Key Exchange
6. Revoke Key
7. Exit
Select an option: 1
Enter username: Alice
[+] AES Symmetric Key Generated for Alice.
Key: K2/ziNu3hCz/c6xXBZbi42yUFl42EQ0oSLBsfwT6A0s=

1. Generate Symmetric Key
2. Encrypt & Decrypt Message (AES)
3. Generate RSA Keys
4. Encrypt & Decrypt Message (RSA)
5. Perform Diffie-Hellman Key Exchange
6. Revoke Key
7. Exit
Select an option: 2
Enter username: Alice
Enter message to encrypt: Attack on Monday
[+] Encrypted Message: yZ5D5QirJZVeXt6iQJBZwDD8LOw5kLsvHd/j7LALXo/J5f5eGDfMsYDO1wByjY18
[+] Decrypted Message: Attack on Monday

1. Generate Symmetric Key
2. Encrypt & Decrypt Message (AES)
3. Generate RSA Keys
4. Encrypt & Decrypt Message (RSA)
5. Perform Diffie-Hellman Key Exchange
6. Revoke Key
7. Exit
Select an option: 3
Enter username: Bob
[+] RSA Key Pair Generated for Bob.
Public Key:
-----BEGIN PUBLIC KEY-----
